<h3>data/temperature/*.csv -> result/temperature/result.csv</h3>
<ol style="color: blue">
  <li>Remove empty lines </li>
  <li>Replace date to year </li>
  <li>Remove cells with older than 1990</li>
  <li>Groups by country+year and sum averages for temperatures</li>
</ol>

In [234]:
! cat data/temperature/*.csv | awk -f com/temperature/removeEmpty.awk | awk -f com/temperature/replaceYearDate.awk | awk -f com/temperature/filterYears.awk | awk -f com/temperature/groupByYearAverageSum.awk  > result/temperature/grouped.csv; awk -f com/countColumsRows.awk result/temperature/grouped.csv 

Columns: 4
Rows: 5785


<h3>data/energy/*.csv -> result/energy/result.csv</h3>
<ol style="color: blue">
  <li>Merge all countries </li>
  <li>Remove cells with older than 1990</li>
</ol>

In [319]:
! awk -f com/energy/merge.awk data/energy/*.csv | awk -f com/energy/filterYears.awk  > result/energy/merged.csv; awk -f com/countColumsRows.awk result/energy/merged.csv

Columns: 128
Rows: 7292


<h3>result/temperature/grouped.csv -> temperature_key.csv</h3>
<ol style="color: blue">
  <li>Makes temporary file which: Concates 4(country) And 1(year) columns and makes (key) column: key="country""year"</li>
</ol>

In [328]:
!cat result/temperature/grouped.csv | mlr --nidx --fs "," put -S '$key="key"; $key=NR == 1 ? "key" : $4.$1' > temperature_key.csv; awk -f com/countColumsRows.awk temperature_key.csv

Columns: 5
Rows: 5785


<h3>result/energy/merged.csv -> energy_key.csv</h3>
<ol style="color: blue">
  <li>Makes temporary file which: Concates 2(country) And 3(year) columns and makes (key) column: key="country""year"</li>
</ol>

In [329]:
!cat result/energy/merged.csv | mlr --nidx --fs "," put -S '$key="key"; $key=NR == 1 ? "key" : $2.$3' > energy_key.csv; awk -f com/countColumsRows.awk energy_key.csv 

Columns: 129
Rows: 7292


<h3>temperature_key.csv  energy_key.csv -> result/temperature_energy_joined.csv</h3>
<ol style="color: blue">
  <li>Join by key column</li>
  <li>Removes key column</li>
</ol>

In [330]:
! mlr --csv join -j key -f temperature_key.csv energy_key.csv | awk -F , 'BEGIN {OFS=FS}  {$1=""; sub(",", ""); print}' > result/temperature_energy_joined.csv; rm temperature_key.csv energy_key.csv; awk -f com/countColumsRows.awk result/temperature_energy_joined.csv

Columns: 132
Rows: 4460


<h3>Select columns and rows</h3>

In [360]:
!awk -F, 'OFS=","{if ($4 == "Afghanistan") print $1, $2}' result/temperature_energy_joined.csv

1990,14.9933
1991,14.3708
1992,14.0561
1993,14.4392
1994,14.7547
1995,14.8592
1996,14.426
1997,14.904
1998,15.1325
1999,15.3887
2000,15.4978
2001,15.7781
2002,15.5377
2003,14.916
2004,15.7709
2005,14.98
2006,15.5956
2007,15.1062
2008,15.2165
2009,15.2577
2010,15.8287
2011,15.518
2012,14.4816
2013,16.5336


In [368]:
!mlr --icsv --ips , --opprint cut -f country,year,coal_prod_change_pct result/temperature_energy_joined.csv